In [2]:
# Import necessary libraries
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from api_credentials import client_id, client_secret, redirect_uri

# Function to fetch playlist tracks and return as a DataFrame
def get_playlist_tracks(client_id, client_secret, redirect_uri, playlist_id):
    scope = 'playlist-read-private'
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                                   client_secret=client_secret,
                                                   redirect_uri=redirect_uri,
                                                   scope=scope))
    
    tracks_data = []
    
    # Handle pagination
    results = sp.playlist_tracks(playlist_id)
    while results:
        for item in results['items']:
            track = item['track']
            album_id = track['album']['id']
            album_details = sp.album(album_id)

            track_info = {
                'Artist': ', '.join([artist['name'] for artist in track['artists']]),
                'Song Title': track['name'],
                'Popularity Score': track['popularity'],
                'Record Label': album_details['label'] if 'label' in album_details else 'Not Available'
            }
            tracks_data.append(track_info)

        # Fetch the next page of tracks
        results = sp.next(results) if results['next'] else None

    df = pd.DataFrame(tracks_data)
    df.index = df.index + 1
    return df

# Mapping of string names to playlist IDs
playlist_names = {
    # top 10 global playlists 
    'todays_top_hits' : "37i9dQZF1DXcBWIGoYBM5M",
    # 'top_50_global' : "37i9dQZEVXbMDoHDwVN2tF",
    # 'rap_caviar' : "37i9dQZF1DX0XUsuxWHRQd",
    # 'viva_latino' : "37i9dQZF1DX10zKzsJ2jva",
    # 'rock_classics' : "37i9dQZF1DWXRqgorJj26U",
    # 'baila_reggaeton' : "37i9dQZF1DWY7IeIP1cdjF",
    # 'all_out_2000s' : "37i9dQZF1DX4o1oenSJRJd",
    # 'songs_to_sing_in_the_car' : "37i9dQZF1DWWMOmoXKqHTD",
    # 'all_out_80s' : "37i9dQZF1DX4UtSsGT1Sbe",
    # 'beast_mode': "37i9dQZF1DX76Wlfdnj7AP",

    # # top AU local playlist (not in order, just a sample)
    # 'NMF_df': '37i9dQZF1DWT2SPAYawYcO',
    # 'HHA_df': '37i9dQZF1DWXXs9GFYnvLB',
    # 'FL_df': '37i9dQZF1DX5WTH49Vcnqp',
    # 'A1_df': '37i9dQZF1DX0sDai2F5jCQ',
    # 'DG_df': '37i9dQZF1DX1lp03JVa0o6',
    # 'GP_df': '37i9dQZF1DXboDblu798Pk'
}

# Dictionary to store each playlist DataFrame
playlists_data = {}

# Process each playlist and store the DataFrame in the dictionary
for name, pid in playlist_names.items():
    playlists_data[name] = get_playlist_tracks(client_id, client_secret, redirect_uri, pid)

# Rename columns for each DataFrame in playlists_data
for df in playlists_data.values():
    df.columns = ['artist', 'song_title', 'popularity_score', 'record_label']

# assign playlist dictionary data to standalone dataframes for each playlist 
# nmf_df = playlists_data['NMF_df']
# hha_df = playlists_data['HHA_df']
# fl_df = playlists_data['FL_df']
# a1_df = playlists_data['A1_df']
# dancegen_df = playlists_data['DG_df']
# getpopped_df = playlists_data['GP_df'] 

#global
todays_top_hits = playlists_data['todays_top_hits']
# top_50_global = playlists_data['top_50_global']
# rap_caviar = playlists_data['rap_caviar']
# viva_latino = playlists_data['viva_latino'],
# rock_classics = playlists_data['rock_classics'],
# baila_reggaeton = playlists_data['baila_reggaeton'],
# all_out_2000s = playlists_data['all_out_2000s'],
# songs_to_sing_in_the_car = playlists_data['songs_to_sing_in_the_car']
# all_out_80s = playlists_data['all_out_80s'],
# beast_mode = playlists_data['beast_mode'],
 

### Hopefully get a larger unique labels list from adding playlists. Ensure all UMG labels are covered for proper analysis. 

In [6]:
# List to hold unique record labels from all dataframes
unique_labels = []

# Iterate over each dataframe in the dictionary
for df_name, df in playlists_data.items():
    # Check if 'record_label' column exists in the dataframe
    if 'record_label' in df.columns:
        # Extract unique labels and add them to the list
        unique_labels.extend(df['record_label'].unique())

# Remove duplicates and create a final list of unique record labels
final_unique_labels = list(set(unique_labels))

# If you want to sort the final list
final_unique_labels.sort()

print(len(final_unique_labels))


39


In [7]:
print(final_unique_labels)

["'94 Sounds/RCA Records", '10K Projects/Capitol Records', '300 Entertainment', 'ADOR', 'Atlantic Records', 'BIGHIT MUSIC', 'Cactus Jack/Epic', 'Columbia', 'DALE PLAY Records', 'Darkroom/Interscope Records', 'Dead Oceans', 'Double P Records', 'Dreamville/Interscope', 'FAX Records/Epic', 'Geffen Records', 'Generation Now/Atlantic', 'Hot Girl Productions', 'Interscope Records', 'Island Records', 'Kemosabe Records/RCA Records', 'Kygo/RCA Records', 'Lions Gate Entertainment Inc. under exclusive license to Geffen Records', 'Mavin Records / Jonzing World', 'Mercury Records/Republic Records', 'Ministry of Sound Recordings', 'Miserable Music Group, LLC', 'Night Street Records, Inc./Warner Records Inc.', 'OVO/Republic', 'Olivia Rodrigo PS', 'RCA Records Label', 'Republic Records', 'Rimas Entertainment LLC', 'Slaughter Gang, LLC/Epic Records', 'Taylor Swift', 'Top Dawg Entertainment/RCA Records', 'UMLE - Latino', 'Warner Records', 'Warner Records/Major Recordings', 'XO / Republic Records']


In [9]:
todays_top_hits['record_label'].unique().tolist()

['Generation Now/Atlantic',
 'Republic Records',
 'Top Dawg Entertainment/RCA Records',
 'Geffen Records',
 'RCA Records Label',
 'XO / Republic Records',
 'Warner Records',
 'Kemosabe Records/RCA Records',
 'Slaughter Gang, LLC/Epic Records',
 'FAX Records/Epic',
 'Mercury Records/Republic Records',
 'Taylor Swift',
 'Warner Records/Major Recordings',
 'OVO/Republic',
 'Island Records',
 'Darkroom/Interscope Records',
 "'94 Sounds/RCA Records",
 'Dreamville/Interscope',
 'Olivia Rodrigo PS',
 'Night Street Records, Inc./Warner Records Inc.',
 'Dead Oceans',
 'Mavin Records / Jonzing World',
 'DALE PLAY Records',
 'Hot Girl Productions',
 'Atlantic Records',
 'BIGHIT MUSIC',
 'Double P Records',
 'Interscope Records',
 'Lions Gate Entertainment Inc. under exclusive license to Geffen Records',
 'Ministry of Sound Recordings',
 '10K Projects/Capitol Records',
 'Kygo/RCA Records',
 'Rimas Entertainment LLC',
 'ADOR',
 'UMLE - Latino',
 '300 Entertainment',
 'Cactus Jack/Epic',
 'Columbia'

In [10]:
# List of record labels to keep
labels_to_keep = ["Universal", "Interscope", "Polydor", "Mercury", "Republic", "Capitol", "Island", "Decca", "Def Jam", "Taylor Swift", "Olivia Rodrigo", "UMC (Universal Music Catalogue)", "UME - Global Clearing House", "UMLE - Latino", "UMe Direct 2", "UNI/MOTOWN", "UNIDISC MUSIC INC.", "Roc Nation"]
pattern = '|'.join(labels_to_keep)


# Dictionary to store filtered and sorted DataFrames
filtered_playlists_data = {}

# Process each playlist DataFrame
for name, df in playlists_data.items():
    # Filter the DataFrame based on record label
    filtered_df = df[df['record_label'].str.contains(pattern, case=False, na=False)]

    # Sort the DataFrame by index
    filtered_df_sorted = filtered_df.sort_index()

    # Store the sorted DataFrame
    filtered_playlists_data[name] = filtered_df_sorted

    # Optionally, print out some stats
    print(f"{name}: UMG has {len(filtered_df_sorted)} out of {len(df)} songs")

# Now, each DataFrame in filtered_playlists_data is filtered and sorted

todays_top_hits: UMG has 15 out of 50 songs


/var/folders/px/x65fp4pn31s5cv4gsbh_3cl80000gn/T/ipykernel_87410/1903073965.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[df['record_label'].str.contains(pattern, case=False, na=False)]


## New Music Friday - UMG tracks & positions 

In [ ]:
# checking the UMG tracks in New Music Friday AU & NZ
filtered_playlists_data['NMF_df']

## Hot Hits Australia - UMG tracks & positions 

In [ ]:
filtered_playlists_data['HHA_df']

,artist,song_title,popularity_score,record_label
1,Noah Kahan,Stick Season,97,Mercury Records/Republic Records
5,Ariana Grande,"yes, and?",97,Republic Records
8,Sabrina Carpenter,Feather,93,Island Records
13,"JID, 21 Savage, Baby Tate",Surround Sound (feat. 21 Savage & Baby Tate),94,Dreamville/Interscope
17,Taylor Swift,Cruel Summer,99,Taylor Swift
22,"The Weeknd, Playboi Carti, Madonna",Popular (with Playboi Carti & Madonna) - From ...,96,XO / Republic Records
26,"Drake, Sexyy Red, SZA",Rich Baby Daddy (feat. Sexyy Red & SZA),93,OVO/Republic
30,Post Malone,Chemical,87,Mercury Records/Republic Records
38,Olivia Rodrigo,vampire,95,Olivia Rodrigo PS
39,Morgan Wallen,Last Night,84,Big Loud Records / Mercury Records / Republic ...


## Front Left - UMG tracks & positions

In [ ]:
filtered_playlists_data['FL_df']

,artist,song_title,popularity_score,record_label
4,"Noah Kahan, Sam Fender",Homesick (with Sam Fender),82,Mercury Records/Republic Records
5,"San Cisco, Nicholas Allbrook",Summer Days,44,Island City Records
16,Lola Young,Wish You Were Dead,62,Universal-Island Records Ltd.
21,"Mumford & Sons, Pharrell Williams",Good People,71,Universal-Island Records Ltd.
26,Still Woozy,Anyone But You,69,"Still Woozy Productions Inc., under exclusive ..."
30,The Last Dinner Party,Caesar on a TV Screen,67,Universal-Island Records Ltd.
34,Tierra Whack,SHOWER SONG,53,Interscope Records
40,Matt Corby,Desert Land,47,Universal Music Australia Pty. Ltd.
47,Lana Del Rey,"Take Me Home, Country Roads",78,Polydor Records
62,Tkay Maidza,Won One,42,Universal Music Australia Pty. Ltd.


## A1 - UMG tracks & positions

In [ ]:
filtered_playlists_data['A1_df']

,artist,song_title,popularity_score,record_label
3,Ice Spice,Think U The Sh*t (Fart),68,10K Projects/Capitol Records
6,Kobie Dee,Statistic,43,Universal Music Australia Pty. Ltd.
7,"Lyrical Lemonade, Lil Tecca, The Kid LAROI, Li...","This My Life (with Lil Tecca, The Kid LAROI & ...",71,Lyrical Lemonade P&D/ Def Jam
12,"JID, 21 Savage, Baby Tate",Surround Sound (feat. 21 Savage & Baby Tate),94,Dreamville/Interscope
14,"Young Franco, S1mba",HITA,52,Universal Music Australia Pty. Ltd.
17,Nicki Minaj,FTCU,91,Republic Records
18,"Coi Leray, Mike WiLL Made-It",Wanna Come Thru,62,Island Records
24,"Drake, J. Cole",Evil Ways (feat. J. Cole),74,OVO/Republic
26,"Nicki Minaj, J. Cole",Let Me Calm Down (feat. J. Cole),76,Republic Records
35,"Bas, J. Cole",Home Alone (with J. Cole),71,Dreamville/Interscope


## Dance Generation - UMG tracks & positions

In [ ]:
filtered_playlists_data['DG_df']

,artist,song_title,popularity_score,record_label
1,Lance Savali,we don't sleep,50,Universal Music New Zealand Limited
7,Sophie Ellis-Bextor,Murder On The Dancefloor,93,Polydor Records
15,"Becky Hill, Sonny Fodera",Never Be Alone,72,Polydor Records
16,"Kungs, David Guetta, Izzy Bizu",All Night Long,77,Universal Music Division Island Def Jam
20,Ariana Grande,"yes, and?",97,Republic Records
22,"Jax Jones, RobbieG",Forever,62,Polydor Records
47,"Skepta, Amy Winehouse",Can't Play Myself (A Tribute To Amy),69,Universal-Island Records Ltd.
54,Ukiyo,attached - demo,41,Universal Music Australia Pty. Ltd.
56,"R3HAB, Pelican",Loca Loca,64,Island Records
66,"Cheat Codes, Sam Feldt",You Don't Even Know Me,63,Republic Records


## Get Popped! - UMG tracks & positions

In [ ]:
filtered_playlists_data['GP_df']

,artist,song_title,popularity_score,record_label
1,Ariana Grande,"yes, and?",97,Republic Records
2,Sophie Ellis-Bextor,Murder On The Dancefloor,93,Polydor Records
8,Taylor Swift,Cruel Summer,99,Taylor Swift
11,"The Weeknd, Playboi Carti, Madonna",Popular (with Playboi Carti & Madonna) - From ...,96,XO / Republic Records
12,Sabrina Carpenter,Feather,93,Island Records
15,"Becky Hill, Sonny Fodera",Never Be Alone,72,Polydor Records
24,"Reneé Rapp, Megan Thee Stallion",Not My Fault (with Megan Thee Stallion),78,Paramount Pictures/Interscope Records
26,"Jonas Brothers, Bailey Zimmerman",Strong Enough (feat. Bailey Zimmerman),75,Republic Records
30,"Drake, Sexyy Red, SZA",Rich Baby Daddy (feat. Sexyy Red & SZA),93,OVO/Republic
38,Post Malone,Chemical,87,Mercury Records/Republic Records


In [ ]:
print(type(filtered_playlists_data))

<class 'dict'>


In [12]:
filtered_playlists_data['todays_top_hits']

,artist,song_title,popularity_score,record_label
2,Ariana Grande,"yes, and?",97,Republic Records
6,"The Weeknd, JENNIE, Lily-Rose Depp","One Of The Girls (with JENNIE, Lily Rose Depp)",98,XO / Republic Records
11,Noah Kahan,Stick Season,97,Mercury Records/Republic Records
12,Taylor Swift,Cruel Summer,99,Taylor Swift
14,"Drake, Sexyy Red, SZA",Rich Baby Daddy (feat. Sexyy Red & SZA),93,OVO/Republic
15,Sabrina Carpenter,Feather,93,Island Records
17,Billie Eilish,What Was I Made For? [From The Motion Picture ...,95,Darkroom/Interscope Records
19,"JID, 21 Savage, Baby Tate",Surround Sound (feat. 21 Savage & Baby Tate),94,Dreamville/Interscope
21,Olivia Rodrigo,vampire,95,Olivia Rodrigo PS
29,"The Weeknd, Playboi Carti, Madonna",Popular (with Playboi Carti & Madonna) - From ...,96,XO / Republic Records


In [ ]:
filtered_playlists_data['baila_reggaeton']

,artist,song_title,popularity_score,record_label
4,KAROL G,OKI DOKI,81,Bichota/Interscope Records
6,"Gaby Music, Lunay, Luar La L",No Te Quieren Conmigo,88,"Universal Music Latino / Gaby Music, Inc."
9,"Feid, ATL Jacob",LUNA,97,UMLE - Latino
13,"Feid, Young Miko",CLASSY 101,87,UMLE - Latino
14,Bad Gyal,Perdió Este Culo,70,Universal Music Latino/Interscope
15,"KAROL G, Cris Mj, Ryan Castro",UNA NOCHE EN MEDELLÍN - REMIX,79,Bichota/Interscope Records
19,"KAROL G, Peso Pluma",QLONA,93,Bichota/Interscope Records
20,"Feid, Pirlo",CUAL ES ESA,83,UMLE - Latino
21,"Brray, Anuel AA, Chencho Corleone, Jhayco, Rya...",Corazón Roto pt. 3,84,UMLE - Latino
23,"Bad Gyal, Tokischa, Young Miko",Chulo pt.2,87,Universal Music Latino/Interscope


In [ ]:
filtered_playlists_data.keys()

dict_keys(['todays_top_hits', 'top_50_global', 'rap_caviar', 'viva_latino', 'rock_classics', 'baila_reggaeton', 'all_out_2000s', 'songs_to_sing_in_the_car', 'all_out_80s', 'beast_mode', 'NMF_df', 'HHA_df', 'FL_df', 'A1_df', 'DG_df', 'GP_df'])

In [ ]:
# Loop through each key (playlist name) in the dictionary
for playlist_name, df in filtered_playlists_data.items():
    # Save each dataframe to a CSV file
    # The file name is the playlist name with .csv extension
    df.to_csv(f"{playlist_name}.csv", index=False)


In [ ]:
filtered_playlists_data['todays_top_hits']

,artist,song_title,popularity_score,record_label
1,Ariana Grande,"yes, and?",97,Republic Records
4,"The Weeknd, JENNIE, Lily-Rose Depp","One Of The Girls (with JENNIE, Lily Rose Depp)",98,XO / Republic Records
10,Noah Kahan,Stick Season,97,Mercury Records/Republic Records
11,Taylor Swift,Cruel Summer,99,Taylor Swift
14,"Drake, Sexyy Red, SZA",Rich Baby Daddy (feat. Sexyy Red & SZA),93,OVO/Republic
15,Sabrina Carpenter,Feather,93,Island Records
16,Billie Eilish,What Was I Made For? [From The Motion Picture ...,95,Darkroom/Interscope Records
18,Olivia Rodrigo,vampire,95,Olivia Rodrigo PS
20,"JID, 21 Savage, Baby Tate",Surround Sound (feat. 21 Savage & Baby Tate),94,Dreamville/Interscope
28,"The Weeknd, Playboi Carti, Madonna",Popular (with Playboi Carti & Madonna) - From ...,96,XO / Republic Records
